In [1]:
pip install torch torchvision torchaudio --upgrade

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.quantization import QuantStub, DeQuantStub, convert
import numpy as np
import torch
import json
print(torch.backends.quantized.supported_engines)

/Users/mervekilicarslan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


['qnnpack', 'none']


Save weights in binary format for compatibility with CPP library

In [7]:
#Save weights compatible with the cpp library
def save_weights_compatible_with_cpp(model, filepath):
    with open(filepath, 'wb') as f:
        # First, write the total number of parameters
        total_params = sum(p.numel() for p in model.parameters())
        f.write(np.array([total_params], dtype=np.int32).tobytes())

        # Now, write each parameter as a flat array of floats
        for param_tensor in model.parameters():
            # Ensure it's a CPU tensor and then convert to numpy
            param_numpy = param_tensor.data.cpu().numpy().ravel().astype(np.float32)
            f.write(param_numpy.tobytes())


def save_quantized_weights_compatible_with_cpp(model, filepath):
    with open(filepath, 'wb') as f:
        # Write each quantized parameter as a flat array of integers, along with scale and zero-point
        for name, module in model.named_modules():
            if hasattr(module, 'weight'):
                # Extract scale and zero-point
                scale = np.array([module.weight().q_scale()], dtype=np.float32)
                zero_point = np.array([module.weight().q_zero_point()], dtype=np.int32)

                # Extract and convert weights to int8
                weight = module.weight().int_repr().cpu().numpy().ravel().astype(np.int8)

                # Write scale, zero-point, and weight to file
                f.write(scale.tobytes())
                f.write(zero_point.tobytes())
                f.write(weight.tobytes())


In [8]:

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transformations for CIFAR-10
transform_cifar10 = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing images for AlexNet
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizing for CIFAR-10
])

# Loading CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data_cifar', train=True, download=True, transform=transform_cifar10)
test_dataset = datasets.CIFAR10(root='./data_cifar', train=False, download=True, transform=transform_cifar10)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [24]:

import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=False),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=False),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.AvgPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=False),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=False),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [19]:
def train(model: nn.Module, dataloader: DataLoader, cuda=False, q=False):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    # Train LeNet on MNIST
    num_epochs = 1
    for epoch in range(num_epochs):
        total_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images, labels
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Print loss every 100 batches
            if (i + 1) % 500 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {total_loss/100:.4f}")
                total_loss = 0.0
    print('Finished Training')

def test(model: nn.Module, dataloader: DataLoader, cuda=False) -> float:
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data

            if cuda:
              inputs = inputs
              labels = labels

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

1.1 No quantization training
Here the training is done without any quantization.

In [20]:
# Assuming the LeNet model and MNIST DataLoaders are initialized
alexnet = AlexNet()

# Train the model
train(alexnet, train_loader, cuda=True, q=False)

# Test the model
accuracy = test(alexnet, test_loader, cuda=True)
print(f'Test Accuracy: {accuracy}%')

# Save the model weights
save_weights_compatible_with_cpp(lenet, 'alexnet_weights.pth')


Epoch [1/1], Step [500/782], Loss: 11.5131
Finished Training
Test Accuracy: 11.7%


**1.2 post-training quantization**

In [25]:
def load_model(quantized_model, model):
    """ Loads in the weights into an object meant for quantization """
    state_dict = model.state_dict()
    model = model.to('cpu')
    quantized_model.load_state_dict(state_dict)


#torch.save(lenet.state_dict(), 'lenet_full_precision.pth')
torch.backends.quantized.engine = 'qnnpack'

# Instantiate the LeNet model
quant_model = AlexNet()
load_model(quant_model, alexnet)
#lenet_quant.load_state_dict(lenet.state_dict())
quant_model.eval()
# Fuse Conv, bn and relu
#net.fuse_model()

# Specify quantization configuration
quant_model.qconfig = torch.quantization.get_default_qconfig('qnnpack')

# Prepare the model for static quantization
torch.quantization.prepare(quant_model, inplace=True)

# Calibrate the model and collect statistics
for inputs, _ in train_loader:
    quant_model(inputs)

# Convert to a quantized model
torch.quantization.convert(quant_model, inplace=True)
# Print scale and zero-point information
for name, module in quant_model.named_modules():
    if hasattr(module, 'scale') and hasattr(module, 'zero_point'):
        print(f"{name} - Scale: {module.scale}, Zero Point: {module.zero_point}")

    # Training process
#score = test(quant_model, test_loader, cuda=True)
#print(f'Accuracy of the network on the test images: {score}%')
save_quantized_weights_compatible_with_cpp(quant_model, 'postquantization_alexnet.pth')


features.0 - Scale: 0.01641475036740303, Zero Point: 122
features.3 - Scale: 0.005665539763867855, Zero Point: 131
features.6 - Scale: 0.002350541530176997, Zero Point: 124
features.8 - Scale: 0.0010496280156075954, Zero Point: 118
features.10 - Scale: 0.00041619251715019345, Zero Point: 133
classifier.1 - Scale: 0.0002170665975427255, Zero Point: 133
classifier.4 - Scale: 0.0001903424708871171, Zero Point: 127
classifier.6 - Scale: 6.74358889227733e-05, Zero Point: 110


NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [MPS, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

MPS: registered at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:75 [backe

1.3 Quantization Aware Training

In [26]:
torch.backends.quantized.engine = 'qnnpack'
qnet = AlexNet()
qnet.qconfig = torch.quantization.get_default_qat_qconfig('qnnpack')
torch.quantization.prepare_qat(qnet, inplace=True)
train(qnet, train_loader, cuda=True)
qnet = qnet.cpu()
torch.quantization.convert(qnet, inplace=True)
#score = test(qnet, test_loader, cuda=False)
#print(f'Accuracy of the fused and quantized network (trained quantized) on the test images:- INT8{score}')
save_quantized_weights_compatible_with_cpp(qnet, 'qat_lenet.pth')



Epoch [1/1], Step [500/782], Loss: 11.5129
Finished Training


NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [MPS, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

MPS: registered at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:75 [backe